# Create an agent with adk, deploy it to Agent Engine  



## Intro
The purpose of this notebook is to have a "one-pager" where you create a simple agent, run it and then deploy it to Agent Engine.  

The relevant documentation kan be found there: https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview

With this the Notebook, you can  
- Explore the main parts of working with ADK  
- Create agents and explore how it shows up in the cloud console, look at the logs etc  
- all implementation steps are "naiv - happy path" to not clutter the code

**Pre-requisites**: Access to a Google Cloud Project  
**Resources created**:
- 1 GCS bucket
- the agents in Agent Engine  

Note:
1. You will need to release the session from an agent to be able to delete it
1. The easiest way to clean up is to use a fresh sandbox project that you can delete after

**Possible next steps**
- Rebuilt the same in VSCode and find out how to manage environments, folder strucure, import etc
- Use the API instead of the SDK as demo-ed here
- Publish it to Agentspace
- Create more advance agents with MCP, custom tools, multi-agents etc

ADK python client doc: https://google.github.io/adk-docs/api-reference/python/index.html  
VertexAI client doc: https://pypi.org/project/google-cloud-aiplatform/    

## Authentication
Mostly for Google Colab.  
You will be authenticated with your user in Colab Enterprise (Vertex AI)

In [ ]:
import sys

if "google.colab" in sys.modules:
    print("Running in Google Colab environment")
    from google.colab import auth
    auth.authenticate_user()

## Configuration

In [ ]:
!pip install --upgrade "google-cloud-aiplatform[adk,agent_engines]==1.106.0"

In [ ]:
# @title Settings
#@markdown ---
#@markdown ### Configure your settings below:

#@markdown _What is the ID of your Google Cloud project._

PROJECT_ID = "my-project-jn3xifcv" #@param {type:"string"}
#@markdown _The Google Cloud region you are using_
LOCATION = "us-central1"  #@param {type:"string"}
#@markdown _The model to use is_
MODEL = "gemini-2.0-flash-001" #@param ["gemini-2.0-pro-exp-02-05","gemini-1.5-pro-002","gemini-1.5-flash-001","gemini-2.0-flash-001"]
#@markdown _The name for your agent in Agent Engine_
AE_APP_NAME = "A secret agent"  #@param {type:"string"}

import os
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["AE_APP_NAME"] = AE_APP_NAME


## Divers init
- enable Vertex AI API
- pip install for AI platform & ADK
- Init Vertex AI


In [ ]:
%%bash
#Vertex AI API
echo "Using project: " $GOOGLE_CLOUD_PROJECT
gcloud config set project $GOOGLE_CLOUD_PROJECT
gcloud services enable aiplatform.googleapis.com
gcloud services enable discoveryengine.googleapis.com

gcloud services list --enabled


In [ ]:
import vertexai
from dotenv import load_dotenv

load_dotenv()
vertexai.init(
    project=os.getenv("GOOGLE_CLOUD_PROJECT"),
    location=os.getenv("GOOGLE_CLOUD_LOCATION"),
    staging_bucket="gs://" + os.getenv("GOOGLE_CLOUD_PROJECT")+"-bucket",
)

# Built it: Agent definition
- a minimalistic agent (with a small trait to see that it's working)
- no external dependencies
- using a tool avalable in ADK

In [ ]:
from google.adk import Agent
from google.adk.tools import google_search  # The Google Search tool

root_agent = Agent(
    # name: A unique name for the agent.
    name="google_search_agent",
    # description: A short description of the agent's purpose, so
    # other agents in a multi-agent system know when to call it.
    description="Answer questions using Google Search.",
    # model: The LLM model that the agent will use:
    model="gemini-2.0-flash-001",
    # instruction: Instructions (or the prompt) for the agent.
    instruction="You are an expert researcher. You stick to the facts. Start all answers with <Well, this is an interesting question>",

    # tools: functions to enhance the model's capabilities.
    # Add the google_search tool below.
    tools=[google_search]
)

# Run your agent locally
- The agent will run in the Notebook
- you can update the question if you want to try diffent questions

In [ ]:
# @title User input for the local test {"run":"auto","vertical-output":true}
question = "what is the capital of Neverland?" #@param {type:"string"}

In [ ]:
from vertexai.preview.reasoning_engines import AdkApp

# Create a simple ADK app for the root_agent.
my_agent =AdkApp(
    agent=root_agent,
    enable_tracing=True
)

# We need a session to use the agent
local_session = my_agent.create_session(user_id="user1")

print("[question] " + question)
# Stream query to get the answer
# Naively print the text of the 1st part of the answer (assuming it exists)
for event in my_agent.stream_query(
    user_id="user1",
    session_id=local_session.id,
    message=question,
):
   # not a robust way to print the result! should test for each element and aggregate parts into 1 answer
   print("[local test] " + event["content"]["parts"][0]["text"])


# Deploy to Agent Engine

If you want to run this several times, change the app name in the setting section above.  

This will  
1. pack agent and dependencies to a GCS bucket  
1. Create the agent engine a.k.a. a reasoning engine  
1. Deploy the agent to Agent engine  
1. To check in the cloud console
  1. Navigate to Vertex AI > Agent Engine (in the Agent Builder section)    
  1. Select one of the deployed agent to see details  
  1. explore the "Deployment details" link in the top of the screen  

In [ ]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    display_name=os.getenv("AE_APP_NAME", "my agent App"),
    agent_engine=root_agent,
    extra_packages=[],
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"
    ]
)
print(remote_app)

## List the agents running in Agent engine



The human friendly name is called Display name, while the machine friendly id is called name and is in the form of  *1234567890123456789*  
The full name of an agent in Agent Engine is in the form _projects/**project-id**/locations/**region name**/reasoningEngines/**name**_  
Disc: earlier versions used the project number and not the project id - it "seems" that it's ok to use the project ID.
The name is also part of the query URL  

In [ ]:

# list all agents
ae_apps = agent_engines.list()

print("Display name | name")
for app in ae_apps:
    print(app.display_name + " | " + app.name)

# finding apps by name:
# ae_apps = agent_engines.list(filter=f'display_name="{app_name}"')
# remote_app = next(ae_apps)


## Query your agent running in Agent Engine




In [ ]:
print("[question] " + question)

# Note: reusing the last deployed app, alt you could find the app by filtrering by name
# finding apps by name:
# ae_apps = agent_engines.list(filter=f'display_name="{app_name}"')
# remote_app = next(ae_apps)

# Get a session for the remote app
remote_session = remote_app.create_session(user_id="u_456")

# Stream query to get the answer
# Naively print the text of the 1st part of the answer (assuming it exists)
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message=question,
):
   # not a robust way to print the result! should test for each element and aggregate parts into 1 answer
   print("[Agent engine] " + event["content"]["parts"][0]["text"])


## Delete agents

To delete an agent, you can either:  
 - use a reference if you have it. Using force=true to delete the child resources like sessions & memory   


In [ ]:
print("deleting agent: " + remote_app.display_name)
remote_app.delete(force=True)

 - or alt. delete from the agent engine by using the agent name.

 Note that when using the agent engine, you will be able to delete an agent in any project you have access to.  

In [ ]:
print("Using Agent engine to delete an agent by name ")
agent_engines.delete("projects/my-project-jn3xifcv/locations/us-central1/reasoningEngines/12345678912345678")


# Clean-up

1. Delete the project if this was a sandbox
2. if you want to keep the project, you will need to
  1. delete the agent(s) (using force=true to delete the child resources like sessions & memory)
  1. delete the GCS bucket

